In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import re

In [166]:
csv_file=open("products_catalogue.csv","w")
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Title',"Link","Description","Remote_testing","Adaptive_Testing","Job_levels","Language","Duration"])

85

In [168]:
products=[]

In [172]:
for i in range(0,133,12):
        url=f"https://www.shl.com/solutions/products/product-catalog/?start={i}&type={2}&type={2}"
        source=requests.get(url).text
        soup=BeautifulSoup(source,'lxml')
        for tr in soup.find_all("tr")[1:]:
                products.append(tr)
    
    

In [173]:
products[:1]

[<tr data-course-id="57">
 <td class="custom__table-heading__title">
 <a href="/solutions/products/product-catalog/view/account-manager-solution/">Account Manager Solution </a>
 </td>
 <td class="custom__table-heading__general">
 <span class="catalogue__circle -yes"></span>
 </td>
 <td class="custom__table-heading__general">
 <span class="catalogue__circle -yes"></span>
 </td>
 <td class="custom__table-heading__general product-catalogue__keys">
 <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">C</span>
 <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">P</span>
 <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">A</span>
 <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">B</span>
 </td>
 </tr>]

In [174]:
for i in range(0,373,12):
        
        url=f"https://www.shl.com/solutions/products/product-catalog/?start={i}&type={1}&type={1}"
        source=requests.get(url).text
        soup=BeautifulSoup(source,'lxml')
        for tr in soup.find_all("tr")[1:]:
                products.append(tr)
        

In [175]:
products=set(products)

In [176]:
len(products)

303

In [177]:
for tr in products:
    try:
        # Extract link and title safely
        anchor = tr.find('a')
        if not anchor or "href" not in anchor.attrs:
            continue  # skip if no link

        link = "https://www.shl.com" + anchor["href"]
        title = anchor.get_text(strip=True)

        # Fetch subpage
        response = requests.get(link)
        if response.status_code != 200:
            continue  # skip if page doesn't load properly

        soup = BeautifulSoup(response.text, 'lxml')
        x = soup.find_all("div", class_="product-catalogue-training-calendar__row typ")

        # Safely extract fields from the subpage
        def get_text_or_na(index):
            try:
                return x[index].p.get_text(strip=True)
            except (IndexError, AttributeError):
                return "N/A"

        description = get_text_or_na(0)
        job_levels = get_text_or_na(1)
        language = get_text_or_na(2)
        duration = get_text_or_na(3)

        # Check for <span> inside table headings
        td = tr.find_all(class_="custom__table-heading__general")

        def has_span(index):
            try:
                return "yes" if td[index].find("span") else "no"
            except IndexError:
                return "no"

        remote_testing = has_span(0)
        adaptive_testing = has_span(1)

        # Prepare data row
        total = [title, link, description,  remote_testing, adaptive_testing,job_levels, language, duration,]
        csv_writer.writerow(total)

    except Exception as e:
        print(f"Error processing a product row: {e}")
        continue  # move to next row safely

   
   
    
    

In [184]:
import pandas as pd

In [208]:
df=pd.read_csv("products_catalogue.csv",encoding='ISO-8859-1')

In [210]:
df.isnull().sum()

Title                0
Link                 0
Description          0
Remote_testing       0
Adaptive_Testing     0
Job_levels           0
Language             8
Duration            20
dtype: int64

In [212]:
df=df.fillna(" ")

In [214]:
df.columns

Index(['Title', 'Link', 'Description', 'Remote_testing', 'Adaptive_Testing',
       'Job_levels', 'Language', 'Duration'],
      dtype='object')

In [216]:
df.isnull().sum()

Title               0
Link                0
Description         0
Remote_testing      0
Adaptive_Testing    0
Job_levels          0
Language            0
Duration            0
dtype: int64

In [218]:
df.to_csv("products_catalogue.csv", index=False)